In [ ]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import LinearRegression


In [ ]:
!pip install pyarrow


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


Q1 - Downloading the data

In [11]:

def read_data(filename:str)->pd.DataFrame:
    """
    Read the data from a parquet file and return a pandas DataFrame.
    """
    # Read the parquet file
    return pd.read_parquet(filename)

jan_data=read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet') 
feb_data=read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet') 







In [15]:
# how many colums are there in the data
jan_data.shape[1]

jan_data.head()


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


Q2. Computing duration

In [16]:
def compute_duration(dataframe:pd.DataFrame)->pd.DataFrame:
    """
    Compute the duration of a trip in seconds.
    """
    dataframe['duration'] = (
        pd.to_datetime(dataframe['tpep_dropoff_datetime']) -
        pd.to_datetime(dataframe['tpep_pickup_datetime'])
    ).dt.total_seconds()/60
    
    return dataframe

jan_data=compute_duration(jan_data)
feb_data=compute_duration(feb_data)



In [17]:
jan_data['duration'].std()

np.float64(42.59435124195458)

Q3. Dropping outliers

In [21]:
desc = jan_data['duration'].describe(percentiles=[0.01,0.05, 0.95, 0.99])   
print(desc.apply(lambda x: f"{x:.2f}" if isinstance(x, float) else f"{x:,}"))

count    3066766.00
mean          15.67
std           42.59
min          -29.20
1%             0.78
5%             3.30
50%           11.52
95%           36.47
99%           57.25
max        10029.18
Name: duration, dtype: object
